In [1]:
from torch.cuda.amp import GradScaler, autocast

/cluster/scratch/goezsoy/miniconda3/envs/cond_text_gen_project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
import numpy as np
import pandas as pd


# read whole corpus
processed_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df.csv')

# work on subset of speakers
temp_bioguides = ['H001055','C001074']
temp_df = processed_df[processed_df.bioguide_id.isin(temp_bioguides)]


/cluster/scratch/goezsoy/miniconda3/envs/cond_text_gen_project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
temp_df

,Unnamed: 0,bioguide_id,speech,first_name,last_name,birthday,gender,term_type,term_start,term_end,term_state,term_party,token_count
40626,41007,C001074,". Mr. Chair, I rise today in support of the am...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,266
40627,41008,C001074,". Madam Chair, I yield 2 minutes to the gentle...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,15
40628,41009,C001074,". Mr. Speaker, I rise today in honor of Nation...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,181
40629,41010,C001074,". Mr. Speaker, I rise today in support of Hous...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,175
40630,41011,C001074,. I yield back the balance of my time. The Ac...,Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,49
40631,41012,C001074,. I thank the gentleman for his remarks. I wou...,Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,94
40632,41013,C001074,. It's a privilege to come to this House floor...,Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,187
40633,41014,C001074,". Mr. Speaker, I rise today to recognize one o...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,130
40634,41015,C001074,". Mr. Speaker, I rise today in support of Nati...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,183
40635,41016,C001074,". Mr. Speaker, I rise on a bit of good news to...",Travis,Childers,1958-03-29,M,H,2009-01-06,2011-01-03,MS,Democrat,185


In [5]:
# loading model for generation
from model import Net

# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = Net().to(device)

checkpoint = torch.load('../checkpoints/speaker_name_as_prompt_deneme_best.pt',map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

cpu


Net(
  (gpt_neo): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0, inplace=False)
      (h): ModuleList(
        (0): GPTNeoBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0, inplace=False)
              (resid_dropout): Dropout(p=0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(in_features=768, out_features=768, bias=False)
              (q_proj): Linear(in_features=768, out_features=768, bias=False)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_features=768, out_features=3072,

In [35]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer('Travis Chil', return_tensors="pt")

gen_tokens = model.gpt_neo.generate(
    inputs.input_ids,
    do_sample=True,
    num_beams=1,
    temperature=0.5,
    max_length=100,
)
tokenizer.batch_decode(gen_tokens)[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Travis Chilas, Mr. Speaker, I rise today to recognize one of America's most exceptional women. Today is indeed a grand day in Prentiss County, Mississippi, because today, Mr. Speaker, Mrs. Myrtis Denson Mayo and all who love her are celebrating 102 years of a life well-lived, one who has sewn every garment she has ever worn. A rich life rewarded by 6 children, 19 grandchildren, 31 great-grandchildren and 20 great-"